# Лаба 6: XGBoost

### Let's Import all necessary libraries

In [1]:
!pip install xlrd xgboost -q

In [2]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, confusion_matrix
from sklearn import metrics
from xgboost import XGBClassifier

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import KFold

In [3]:
import os

# This is my file path for the lab works
os.chdir("C:/Users/bahra/Desktop/ITMO Master's/2nd Semester/АД на Python/Лаба 6")

### Loading the dataset

In [4]:
# Let's read the data
df = pd.read_csv("default of credit card clients.csv", sep = ';')
# Let's see what we have
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default_payment_next_month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


### Let's read it by default payment next month
* We have get a clear view of our data

In [5]:
df.groupby(['default_payment_next_month']).count()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
default_payment_next_month,,,,,,,,,,,,,,,,,,,,,
0,23364,23364,23364,23364,23364,23364,23364,23364,23364,23364,...,23364,23364,23364,23364,23364,23364,23364,23364,23364,23364
1,6636,6636,6636,6636,6636,6636,6636,6636,6636,6636,...,6636,6636,6636,6636,6636,6636,6636,6636,6636,6636


## As you can see, our data is not unbalanced. We can draw several conclusions from this: 

* To assess the quality of models, we cannot use accuracy metrics. We will use a score for unbalanced data -- f1 score 
* Since the training of the XGBoost algorithm is affected by the imbalance of the data, we will generate additional data by copying into the training data

In [6]:
X = df.drop(["ID", "default_payment_next_month"], axis=1)
y = df["default_payment_next_month"]

In [7]:
new_df = df.sample(frac=1)

In [8]:
len(new_df)

30000

In [9]:
df_train = new_df[:int(len(new_df) * 0.7)]
df_test = new_df[int(len(new_df) * 0.7):]

In [10]:
df_train.groupby(['default_payment_next_month']).count()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
default_payment_next_month,,,,,,,,,,,,,,,,,,,,,
0,16402,16402,16402,16402,16402,16402,16402,16402,16402,16402,...,16402,16402,16402,16402,16402,16402,16402,16402,16402,16402
1,4598,4598,4598,4598,4598,4598,4598,4598,4598,4598,...,4598,4598,4598,4598,4598,4598,4598,4598,4598,4598


In [11]:
df_test.groupby(['default_payment_next_month']).count()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
default_payment_next_month,,,,,,,,,,,,,,,,,,,,,
0,6962,6962,6962,6962,6962,6962,6962,6962,6962,6962,...,6962,6962,6962,6962,6962,6962,6962,6962,6962,6962
1,2038,2038,2038,2038,2038,2038,2038,2038,2038,2038,...,2038,2038,2038,2038,2038,2038,2038,2038,2038,2038


In [12]:
df.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default_payment_next_month'],
      dtype='object')

In [13]:
df.describe()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default_payment_next_month
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,...,30000.000000,30000.000000,30000.000000,30000.000000,3.000000e+04,30000.00000,30000.000000,30000.000000,30000.000000,30000.000000
mean,15000.500000,167484.322667,1.603733,1.853133,1.551867,35.485500,-0.016700,-0.133767,-0.166200,-0.220667,...,43262.948967,40311.400967,38871.760400,5663.580500,5.921163e+03,5225.68150,4826.076867,4799.387633,5215.502567,0.221200
std,8660.398374,129747.661567,0.489129,0.790349,0.521970,9.217904,1.123802,1.197186,1.196868,1.169139,...,64332.856134,60797.155770,59554.107537,16563.280354,2.304087e+04,17606.96147,15666.159744,15278.305679,17777.465775,0.415062
min,1.000000,10000.000000,1.000000,0.000000,0.000000,21.000000,-2.000000,-2.000000,-2.000000,-2.000000,...,-170000.000000,-81334.000000,-339603.000000,0.000000,0.000000e+00,0.00000,0.000000,0.000000,0.000000,0.000000
25%,7500.750000,50000.000000,1.000000,1.000000,1.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,2326.750000,1763.000000,1256.000000,1000.000000,8.330000e+02,390.00000,296.000000,252.500000,117.750000,0.000000
50%,15000.500000,140000.000000,2.000000,2.000000,2.000000,34.000000,0.000000,0.000000,0.000000,0.000000,...,19052.000000,18104.500000,17071.000000,2100.000000,2.009000e+03,1800.00000,1500.000000,1500.000000,1500.000000,0.000000
75%,22500.250000,240000.000000,2.000000,2.000000,2.000000,41.000000,0.000000,0.000000,0.000000,0.000000,...,54506.000000,50190.500000,49198.250000,5006.000000,5.000000e+03,4505.00000,4013.250000,4031.500000,4000.000000,0.000000
max,30000.000000,1000000.000000,2.000000,6.000000,3.000000,79.000000,8.000000,8.000000,8.000000,8.000000,...,891586.000000,927171.000000,961664.000000,873552.000000,1.684259e+06,896040.00000,621000.000000,426529.000000,528666.000000,1.000000


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   ID                          30000 non-null  int64
 1   LIMIT_BAL                   30000 non-null  int64
 2   SEX                         30000 non-null  int64
 3   EDUCATION                   30000 non-null  int64
 4   MARRIAGE                    30000 non-null  int64
 5   AGE                         30000 non-null  int64
 6   PAY_0                       30000 non-null  int64
 7   PAY_2                       30000 non-null  int64
 8   PAY_3                       30000 non-null  int64
 9   PAY_4                       30000 non-null  int64
 10  PAY_5                       30000 non-null  int64
 11  PAY_6                       30000 non-null  int64
 12  BILL_AMT1                   30000 non-null  int64
 13  BILL_AMT2                   30000 non-null  int64
 14  BILL_A

### Let's extract data in train

In [15]:
df_1 = df_train[df_train["default_payment_next_month"] == 1]
df_0 = df_train[df_train["default_payment_next_month"] == 0]
df_temp = df_1
m = len(df_0) // len(df_1) 
ost = len(df_0) % len(df_1) 

for i in range(m - 1):
    df_temp = df_temp.append(df_1)
    
df_temp = df_temp.append(df_1.sample(frac=1).head(ost))
df_train = df_temp.append(df_0).sample(frac=1)

C:\Users\bahra\AppData\Local\Temp\ipykernel_23864\343241870.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_temp = df_temp.append(df_1)
C:\Users\bahra\AppData\Local\Temp\ipykernel_23864\343241870.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_temp = df_temp.append(df_1)
C:\Users\bahra\AppData\Local\Temp\ipykernel_23864\343241870.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_temp = df_temp.append(df_1.sample(frac=1).head(ost))
C:\Users\bahra\AppData\Local\Temp\ipykernel_23864\343241870.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_temp.append(df_0).sample(frac=1)


In [16]:
df_train.groupby(['default_payment_next_month']).count()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
default_payment_next_month,,,,,,,,,,,,,,,,,,,,,
0,16402,16402,16402,16402,16402,16402,16402,16402,16402,16402,...,16402,16402,16402,16402,16402,16402,16402,16402,16402,16402
1,16402,16402,16402,16402,16402,16402,16402,16402,16402,16402,...,16402,16402,16402,16402,16402,16402,16402,16402,16402,16402


## Let's divide the data into train and test sample

In [17]:
X_train = df_train.drop(["ID", "default_payment_next_month"], axis=1)
y_train = df_train["default_payment_next_month"]


X_test = df_test.drop(["ID", "default_payment_next_month"], axis=1)
y_test = df_test["default_payment_next_month"]

## XGBoost

In [18]:
model = XGBClassifier(seed=42,
                      n_estimators=100,
                      max_depth=6,
                      learning_rate=0.3)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.3, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

## Grid Search CV with best parameters

In [20]:
f1 = make_scorer(f1_score, average='weighted')

grid_param = {  
    'n_estimators': [100, 500, 1000],
    'max_depth': [3, 4, 6]
}

searcher = GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(seed=1234, learning_rate=0.1),
       n_jobs=-1,
       param_grid=grid_param,
       pre_dispatch='2*n_jobs', refit=True, scoring=f1, verbose=0)

searcher.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None...
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'max_depth': [3, 4, 6],
                         'n_estimators': [100, 500, 1000]},
             scoring=make_scorer(f1_score, average=weighted))

In [21]:
searcher.best_score_

0.9004034271204674

In [22]:
searcher.best_params_

{'max_depth': 6, 'n_estimators': 1000}

In [23]:
searcher.best_estimator_

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=1000, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [24]:
res = (
    pd.DataFrame({
        "mean_test_score": searcher.cv_results_["mean_test_score"],
        "mean_fit_time": searcher.cv_results_["mean_fit_time"]})
      .join(pd.io.json.json_normalize(searcher.cv_results_["params"]).add_prefix("param_"))
)

C:\Users\bahra\AppData\Local\Temp\ipykernel_23864\1294350662.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  .join(pd.io.json.json_normalize(searcher.cv_results_["params"]).add_prefix("param_"))


In [25]:
print(res)

   mean_test_score  mean_fit_time  param_max_depth  param_n_estimators
0         0.720989       2.943083                3                 100
1         0.746798      28.765273                3                 500
2         0.773468      62.982634                3                1000
3         0.726575       4.075612                4                 100
4         0.778715      40.136348                4                 500
5         0.826158      80.463868                4                1000
6         0.756332      15.575129                6                 100
7         0.857185      52.259256                6                 500
8         0.900403      77.180298                6                1000


## Train the best model configuration on all training data 

In [26]:
classifier = searcher.best_estimator_

classifier.fit(X_train, y_train)  

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=1000, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

## Evaluating results

In [ ]:
y_pred_train = classifier.predict(X_train)  
y_pred_test = classifier.predict(X_test)  

In [ ]:
print(u'Training sample contingency table')
conf_mat = confusion_matrix(y_train, y_pred_train)

# Making beautiful
conf_mat_train = pd.DataFrame(conf_mat, index=classifier.classes_, columns=classifier.classes_)
conf_mat_train

In [ ]:
print('F1 score on training set', f1_score(y_train, y_pred_train, average='weighted'))

In [ ]:
print(u'Contingency table for test set')
conf_mat = confusion_matrix(y_test, y_pred_test)

# Making beautiful
conf_mat_train = pd.DataFrame(conf_mat, index=classifier.classes_, columns=classifier.classes_)
conf_mat_train

In [ ]:
print('F1 score on test set', f1_score(y_test, y_pred_test, average='weighted'))

**An important feature of wooden modules is that we can easily analyze the importance of each feature and thus interpret how the models work. This property is the most important advantage of wooden algorithms, since no one needs a black box. We will analyze features using the shap library**

In [ ]:
sns.set(font_scale = 1.5)

In [ ]:
xgb.plot_importance(classifier)

In [ ]:
xgb.plot_importance(model, max_num_features = 10)
#  xgb.plot_importance(model, max_num_features = 20)
#  xgb.plot_importance(model, max_num_features = 30)

In [ ]:
import shap
shap.initjs()

In [ ]:
explainer = shap.TreeExplainer(classifier)
shap_values = explainer.shap_values(X_test[:1000])

In [ ]:
shap.summary_plot(shap_values, X_test[:1000])

**As you can see, the PAY_0 feature is the most significant and informative, the higher the value of this feature, the more likely it is that the model prediction will be 1. Features:**
* SEX
* EDUCATION 
* PAY_2
* PAY_3 
* PAY_4 
* PAY_5 

**were hardly used by the model, so they can be easily discarded if desired and retrained the model without losing quality**